In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
#Download the data

In [20]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [21]:
concrete_data.shape

(1030, 9)

In [22]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [23]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [24]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [25]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [26]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [27]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [28]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

In [29]:
## Build a Neural Network
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(40, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(40, activation='softmax'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=42)

In [31]:
# build the model
model = regression_model()

In [32]:
## Train and Test the Network

# build the model
model.fit( X_test, y_test,epochs=150, verbose=2)

Epoch 1/150
7/7 - 1s - loss: 1519.0342
Epoch 2/150
7/7 - 0s - loss: 1507.5375
Epoch 3/150
7/7 - 0s - loss: 1505.6669
Epoch 4/150
7/7 - 0s - loss: 1504.6106
Epoch 5/150
7/7 - 0s - loss: 1503.5356
Epoch 6/150
7/7 - 0s - loss: 1502.4974
Epoch 7/150
7/7 - 0s - loss: 1501.4489
Epoch 8/150
7/7 - 0s - loss: 1500.4026
Epoch 9/150
7/7 - 0s - loss: 1499.3514
Epoch 10/150
7/7 - 0s - loss: 1498.3239
Epoch 11/150
7/7 - 0s - loss: 1497.3132
Epoch 12/150
7/7 - 0s - loss: 1496.2699
Epoch 13/150
7/7 - 0s - loss: 1495.2644
Epoch 14/150
7/7 - 0s - loss: 1494.2382
Epoch 15/150
7/7 - 0s - loss: 1493.2444
Epoch 16/150
7/7 - 0s - loss: 1492.2482
Epoch 17/150
7/7 - 0s - loss: 1491.2277
Epoch 18/150
7/7 - 0s - loss: 1490.2317
Epoch 19/150
7/7 - 0s - loss: 1489.2369
Epoch 20/150
7/7 - 0s - loss: 1488.2302
Epoch 21/150
7/7 - 0s - loss: 1487.2500
Epoch 22/150
7/7 - 0s - loss: 1486.2555
Epoch 23/150
7/7 - 0s - loss: 1485.2705
Epoch 24/150
7/7 - 0s - loss: 1484.2865
Epoch 25/150
7/7 - 0s - loss: 1483.2993
Epoch 26/

In [33]:

loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

7/7 [==============================] - 0s 1ms/step - loss: 1364.8655


1364.865478515625

In [ ]:
total_mean_squared_errors = 150
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=4)
    model.fit(X_train, y_train, epochs=150, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))




MSE 1: 951.409912109375
MSE 2: 638.3656005859375
MSE 3: 433.8968811035156
MSE 4: 329.8431701660156
MSE 5: 303.11151123046875
MSE 6: 302.90625
MSE 7: 303.05657958984375
MSE 13: 303.0633850097656
MSE 14: 303.0689697265625
MSE 15: 303.06048583984375
MSE 16: 303.0616760253906
MSE 17: 303.0704345703125
MSE 18: 303.0740966796875
MSE 19: 303.0693359375
MSE 20: 303.06549072265625
MSE 21: 303.06536865234375
MSE 22: 303.0644836425781
MSE 23: 303.06793212890625
MSE 24: 303.0616760253906
MSE 25: 303.0675964355469
MSE 26: 303.065185546875
MSE 27: 303.0727844238281
MSE 28: 303.06573486328125
MSE 29: 303.0577697753906
MSE 30: 303.0647888183594
MSE 31: 303.0639343261719
MSE 32: 303.0637512207031
MSE 33: 303.0664367675781
MSE 34: 303.0530700683594
MSE 35: 303.0619812011719
MSE 36: 303.064453125
MSE 37: 303.0729064941406
MSE 38: 303.0647888183594
MSE 39: 303.0606689453125
MSE 40: 303.0588073730469
MSE 41: 303.0703125
MSE 42: 303.0667419433594
MSE 43: 303.0674743652344
MSE 44: 303.0724792480469
MSE 45: 3